# A

In [1]:
C = "a"

In [3]:
print(chr(ord(C) + 1))

b


# B

In [4]:
N, K, M = 5, 10, 7
A = [8, 10, 3, 6]

print(8)

8


In [6]:
ans = N * M - sum(A)
if ans <= K:
    print(max(0, ans))
else:
    print(-1)

8


# C

In [28]:
N, M = 2, 5
X = [['1', 'WA'], ['1', 'AC'], ['2', 'WA'], ['2', 'AC'], ['2', 'WA'], ['2', 'WA']]

print(2, 2)

2 2


In [24]:
N, M = 2, 5
X = [['1', 'WA'], ['1', 'AC'], ['2', 'WA'], ['2', 'WA'], ['2', 'WA'], ['2', 'WA']]

print(2, 2)

2 2


In [29]:
acs = [False] * (N + 1)
was = [0] * (N + 1)

for p, s in X:
    if s == "AC":
        acs[int(p)] = True
    elif not acs[int(p)]:
        was[int(p)] += 1
        
was_cnt = 0
for a, w in zip(acs, was):
    if a:
        was_cnt += w

print(sum(acs), was_cnt)

2 2


# D

In [1]:
H, W = 3, 3
S = ['...', '...', '...']

print(4)

4


In [9]:
H, W = 3, 5
S = ['...#.', '.#.#.', '.#...']

print(10)

10


In [10]:
from collections import deque

def solve(x, y):
    offset = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    visited = [[-1] * W for _ in range(H)]
    visited[y][x] = 0
    
    q = deque()
    q.append((x, y))

    while q:
        x, y = q.popleft()

        for u, v in offset:
            if not (0 <= x + u < W) or not (0 <= y + v < H):
                continue

            if visited[y + v][x + u] > -1 or S[y + v][x + u] == "#":
                continue

            visited[y + v][x + u] = visited[y][x] + 1
            q.append((x + u, y + v))
            
    return visited

In [11]:
ans = 0
for y, ss in enumerate(S):
    for x, s in enumerate(ss):
        if s == ".":
            ans = max(ans, max(max(v) for v in solve(x, y)))

print(ans)

10


# E

In [31]:
N, K = 4, 2
A = [1, 1, 3, 4]

print(11)

11


階乗，及び階乗の逆元

In [32]:
MOD = 10 ** 9 + 7

fac = [0] * (N + 1)
fac[0] = 1
fac[1] = 1
for i in range(2, N + 1):
    fac[i] = fac[i - 1] * i % MOD

finv = [0] * (N + 1)
finv[N] = pow(fac[N], MOD - 2, MOD)
for i in reversed(range(1, N + 1)):
    finv[i - 1] = finv[i] * i % MOD

Combination

In [33]:
def comb(n, k):
    if n < k:
        return 0
    return fac[n] * finv[k] * finv[n - k] % MOD

自分がmax, minそれぞれになる場合の数をかけて，足していく．

In [37]:
ans = 0
for i, a in enumerate(sorted(A, reverse=True)):
    ans += a * comb(N - i - 1, K - 1)
    ans -= a * comb(i, K - 1)
    ans %= MOD
    
print(ans)

11


# F

解答例) http://drken1215.hatenablog.com/entry/2020/01/12/224200

#### 2次元ベクトル

In [2]:
import math

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f"({self.x}, {self.y})"
    
    def __add__(self, q):
        if isinstance(q, Point):
            return Point(self.x + q.x, self.y + q.y)
        else:
            return Point(self.x + q, self.y + q)
    
    def __sub__(self, q):
        if isinstance(q, Point):
            return Point(self.x - q.x, self.y - q.y)
        else:
            return Point(self.x - q, self.y - q)
    
    def __mul__(self, q):
        if isinstance(q, Point):
            return Point(self.x * q.x - self.y * q.y,
                         self.x * q.y + self.y * q.x)
        else:
            return Point(self.x * q, self.y * q)
        
    def __truediv__(self, q):
        if isinstance(q, Point):
            return self * q.conj / q.norm
        else:
            return Point(self.x / q, self.y / q)
    
    @property
    def conj(self):
        return Point(self.x, -self.y)
    
    def rot(self, ang):
        return Point(math.cos(ang) * self.x - math.sin(ang) * self.y,
                     math.sin(ang) * self.x + math.cos(ang) * self.y)
    
    def rot90(self):
        return Point(-self.y, self.x)
    
    def cross(self, q):
        return self.x * q.y - self.y * q.x
    
    def dot(self, q):
        return self.x * q.x + self.y * q.y
    
    @property
    def norm(self):
        return self.dot(self)

    def __abs__(self):
        return self.dot(self) ** 0.5

#### 直線

In [29]:
class Line:
    def __init__(self, a, b):
        if not isinstance(a, Point) or not isinstance(b, Point):
            raise AttributeError("points should be instance of Point")
            
        self.a = a
        self.b = b
        
    def __repr__(self):
        return f"[{self.a}, {self.b}]"
    
    def cross_point(self, l, eps=1e-12):
        d = (self.b - self.a).cross(l.b - l.a)
        if abs(d) < eps:
            return Point(0, 0)
        
        return self.a + (self.b - self.a) * (l.b - l.a).cross(l.b - self.a) / d

#### 3 点 a,b,c が同一直線上にあるかどうかを判定する

In [22]:
def simple_ccw(a, b, c, eps=1e-12):
    if (b - a).cross(c - a) > eps:
        return 1
    if (b - a).cross(c - a) < -eps:
        return -1
    return 0

In [9]:
a = Point(1, 1)
b = Point(3, 3)
c = Point(2, 2)

simple_ccw(a, b, c)

0

#### 外心を求める．

In [25]:
def circum_center(a, b, c):
    ab = Line((a + b) / 2, (a + b) / 2 + (a - b).rot90())
    bc = Line((b + c) / 2, (b + c) / 2 + (b - c).rot90())
    return ab.cross_point(bc)

In [26]:
circum_center(a, b, c)

(0, 0)

## 解法

In [12]:
N = 2
vecs = [Point(0, 0), Point(1, 0)]

print(0.50)

0.5


#### 円の中心の候補を求める．

In [14]:
cand = []
for i in range(N):
    for j in range(i + 1, N):
        cand.append((vecs[i] + vecs[j]) / 2)
        
        for k in range(j + 1, N):
            if simple_ccw(vecs[i], vecs[j], vecs[k]) == 0:
                continue
                
            cand.append(circum_center(vecs[i], vecs[j], vecs[k]))
            
cand

[(0.5, 0.0)]

#### 各候補について，$N$個の点への距離の最大値を求める．

解答は，最大値リストの中の最小値．

In [27]:
ans = 1e12
for p in cand:
    tmp = 0
    for v in vecs:
        tmp = max(tmp, abs(p - v))
    ans = min(ans, tmp)
print(ans)

0.5


## 解法2: 三分探索

In [34]:
def f(x, y):
    res = 0
    for i in range(N):
        res = max(res, abs(vecs[i] - Point(x, y)))
    return res

In [31]:
def g(x, max_iter=100):
    left = -1000
    right = 1000
    for _ in range(max_iter):
        m1 = (left * 2 + right) / 3
        m2 = (left + right * 2) / 3
        if f(x, m1) > f(x, m2):
            left = m1
        else:
            right = m2
            
    return f(x, left)

In [32]:
def solve(max_iter=100):
    left = -1000
    right = 1000
    for _ in range(max_iter):
        m1 = (left * 2 + right) / 3
        m2 = (left + right * 2) / 3
        if g(m1) > g(m2):
            left = m1
        else:
            right = m2
            
    return g(left)

In [35]:
solve()

0.5000000000000021